In [33]:
import tensorflow as tf
from makiflow.models.classificator import Classificator
from makiflow.metrics import bin_categorical_dice_coeff
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import pandas as pd
import json
from tqdm import tqdm
%matplotlib inline

from makiflow import set_main_gpu
set_main_gpu(';')

In [6]:
from makiflow.metrics.utils import one_hot

In [2]:
N_CLASSES = 8
IMAGE_SHAPE = (1024, 1024)

In [3]:
SESSION = tf.Session()

In [122]:
def create_model(arch_path, weights_path):
    model = Classificator.from_json(arch_path)
    model.set_session(SESSION)
    model.load_weights(weights_path)
    return model

def load_im_mask(image_path, masks_folder_path):
    image_shape = IMAGE_SHAPE
    n_classes = N_CLASSES
    image = cv2.imread(image_path)

    mask_folder = masks_folder_path

    label_tensor = np.zeros(shape=(*image_shape, n_classes + 1), dtype='int32')
    for binary_mask_path in glob(os.path.join(mask_folder, '*')):
        filename = binary_mask_path.split('/')[-1]
        class_id = int(filename.split('.')[0])
        assert class_id != 0, 'Encountered class 0. Class names must start from 1.'
        binary_mask = cv2.imread(binary_mask_path)
        assert binary_mask is not None, f'Could not load mask with name={binary_mask_path}'
        label_tensor[..., class_id] = binary_mask[..., 0] * class_id
    label_tensor = np.max(label_tensor, axis=-1)
    
    return image.astype(np.float32, copy=False) / 255, label_tensor

def load_data(config_path):
    config = load_json(config_path)
    config = config['test_config']
    image_paths = config['test_image']
    mask_paths = config['test_mask']
    
    images = []
    masks = []
    for image_path, mask_folder_path in zip(image_paths, mask_paths):
        image, mask = load_im_mask(image_path, mask_folder_path)
        images.append(image)
        masks.append(mask)
    return images, masks

def load_json(path):
    with open(path, 'r') as f:
        return json.loads(f.read())

def test(arch_path, weights_path, config_path, th=0.5):
    images, masks = load_data(config_path)
    model = create_model(arch_path, weights_path)
    
    predictions = []
    max_preds = []
    for image in tqdm(images):
        pred = (model.predict([image])[0] > th).astype('uint8') * np.arange(1, 9, 1).reshape(1, 1, 8)
#         pred = pred.max(axis=-1)
        predictions += [pred]
        max_preds += [pred.max(axis=-1)]
    
    # predictions = np.concatenate(predictions, axis=0)
    f1_scores = f1_score(np.asarray(masks).reshape(-1), np.asarray(max_preds).reshape(-1), average=None)
    return confusion_matrix(np.asarray(masks).reshape(-1), np.asarray(max_preds).reshape(-1)), f1_scores


def compute_tpr_tnr_class(class_id, C):
    TN = C[0, 0]
    TP = C[class_id, class_id]
    
    FP = 0
    for i in range(0, C.shape[0]):
        if i == class_id:
            continue
        FP += C[i, class_id]
        
    FN = 0
    for i in range(0, C.shape[0]):
        if i == class_id:
            continue
        FN += C[class_id, i]
    
    tpr = 1 - FN / (FN + TP + 1e-7)
    tnr = 1 - FP / (FP + TN + 1e-7)
    acc = (TP + TN) / (TP + TN + FP + FN)
    return tpr, tnr, acc
    
def compute_tpr(C):
    trps = []
    tnrs = []
    accs = []
    for i in range(1, C.shape[0]):
        rates = compute_tpr_tnr_class(i, C)
        trps.append(rates[0])
        tnrs.append(rates[1])
        accs.append(rates[2])
    return trps, tnrs, accs

In [107]:
cv3_f0_model_arch = 'cv3_fold0/result/model.json'
cv3_f0_model_weights = 'cv3_fold0/result/last_weights/weights.ckpt'
cv3_f0_config = 'cv3_fold0/config.json'

cv3_f0_C, f1_scores_1 = test(cv3_f0_model_arch, cv3_f0_model_weights, cv3_f0_config, 0.5)

Model is restored!
INFO:tensorflow:Restoring parameters from cv3_fold0/result/last_weights/weights.ckpt


  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Weights are loaded.



1it [00:07,  7.84s/it]
 12%|█▎        | 1/8 [00:07<00:55,  7.93s/it]
0it [00:00, ?it/s]
1it [00:03,  3.23s/it]
 25%|██▌       | 2/8 [00:11<00:39,  6.54s/it]
0it [00:00, ?it/s]
1it [00:03,  3.24s/it]
 38%|███▊      | 3/8 [00:17<00:31,  6.32s/it]
0it [00:00, ?it/s]
1it [00:03,  3.17s/it]
 50%|█████     | 4/8 [00:23<00:25,  6.34s/it]
0it [00:00, ?it/s]
1it [00:03,  3.23s/it]
 62%|██████▎   | 5/8 [00:30<00:19,  6.42s/it]
0it [00:00, ?it/s]
1it [00:03,  3.17s/it]
 75%|███████▌  | 6/8 [00:36<00:12,  6.47s/it]
0it [00:00, ?it/s]
1it [00:03,  3.17s/it]
 88%|████████▊ | 7/8 [00:43<00:06,  6.68s/it]
0it [00:00, ?it/s]
1it [00:03,  3.25s/it]
100%|██████████| 8/8 [00:51<00:00,  6.38s/it]


In [123]:
tpr0, tnr0, accs0 = np.array(compute_tpr(cv3_f0_C)).round(2)

In [111]:
tpr0, tnr0

(array([0.98, 0.64, 0.92, 0.89, 0.28, 0.43, 0.17, 0.87]),
 array([0.97, 0.99, 0.72, 0.98, 1.  , 1.  , 1.  , 0.95]))

In [64]:
cv3_f1_model_arch = 'cv3_fold1/result/model.json'
cv3_f1_model_weights = 'cv3_fold1/result/last_weights/weights.ckpt'
cv3_f1_config = 'cv3_fold1/config.json'

cv3_f1_dice, f1_scores_2 = test(cv3_f1_model_arch, cv3_f1_model_weights, cv3_f1_config, 0.5)

Model is restored!
INFO:tensorflow:Restoring parameters from cv3_fold1/result/last_weights/weights.ckpt


  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Weights are loaded.



1it [00:07,  7.25s/it]
 12%|█▎        | 1/8 [00:07<00:51,  7.35s/it]
0it [00:00, ?it/s]
1it [00:03,  3.01s/it]
 25%|██▌       | 2/8 [00:10<00:36,  6.07s/it]
0it [00:00, ?it/s]
1it [00:03,  3.13s/it]
 38%|███▊      | 3/8 [00:14<00:27,  5.42s/it]
0it [00:00, ?it/s]
1it [00:02,  2.86s/it]
 50%|█████     | 4/8 [00:20<00:22,  5.63s/it]
0it [00:00, ?it/s]
1it [00:03,  3.19s/it]
 62%|██████▎   | 5/8 [00:24<00:15,  5.06s/it]
0it [00:00, ?it/s]
1it [00:03,  3.04s/it]
 75%|███████▌  | 6/8 [00:28<00:09,  4.70s/it]
0it [00:00, ?it/s]
1it [00:03,  3.03s/it]
 88%|████████▊ | 7/8 [00:32<00:04,  4.75s/it]
0it [00:00, ?it/s]
1it [00:02,  2.98s/it]
100%|██████████| 8/8 [00:38<00:00,  4.82s/it]


In [124]:
tpr1, tnr1, accs1 = np.array(compute_tpr(cv3_f1_dice)).round(2)

In [113]:
tpr1, tnr1

(array([0.99, 0.86, 0.91, 0.78, 0.82, 0.14, 0.64, 0.92]),
 array([0.98, 0.98, 0.74, 0.99, 1.  , 1.  , 0.99, 0.95]))

In [67]:
cv3_f2_model_arch = 'cv3_fold2/result/model.json'
cv3_f2_model_weights = 'cv3_fold2/result/last_weights/weights.ckpt'
cv3_f2_config = 'cv3_fold2/config.json'

cv3_f2_dice, f1_scores_3 = test(cv3_f2_model_arch, cv3_f2_model_weights, cv3_f2_config, 0.5)

Model is restored!
INFO:tensorflow:Restoring parameters from cv3_fold2/result/last_weights/weights.ckpt


  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Weights are loaded.



1it [00:07,  7.23s/it]
 12%|█▎        | 1/8 [00:07<00:51,  7.32s/it]
0it [00:00, ?it/s]
1it [00:03,  3.04s/it]
 25%|██▌       | 2/8 [00:12<00:39,  6.62s/it]
0it [00:00, ?it/s]
1it [00:03,  3.09s/it]
 38%|███▊      | 3/8 [00:17<00:31,  6.28s/it]
0it [00:00, ?it/s]
1it [00:02,  2.96s/it]
 50%|█████     | 4/8 [00:23<00:24,  6.01s/it]
0it [00:00, ?it/s]
1it [00:03,  3.00s/it]
 62%|██████▎   | 5/8 [00:28<00:17,  5.86s/it]
0it [00:00, ?it/s]
1it [00:02,  2.94s/it]
 75%|███████▌  | 6/8 [00:34<00:11,  5.76s/it]
0it [00:00, ?it/s]
1it [00:03,  3.13s/it]
 88%|████████▊ | 7/8 [00:39<00:05,  5.64s/it]
0it [00:00, ?it/s]
1it [00:02,  2.88s/it]
100%|██████████| 8/8 [00:45<00:00,  5.64s/it]


In [125]:
tpr2, tnr2, accs2 = np.array(compute_tpr(cv3_f2_dice)).round(2)

In [85]:
f1_scores_1 = f1_scores_1[1:].round(2)
f1_scores_1 = np.hstack([f1_scores_1, np.mean(f1_scores_1).round(2)])
f1_scores_2 = f1_scores_2[1:].round(2)
f1_scores_2 = np.hstack([f1_scores_2, np.mean(f1_scores_2).round(2)])
f1_scores_3 = f1_scores_3[1:].round(2)
f1_scores_3 = np.hstack([f1_scores_3, np.mean(f1_scores_3).round(2)])

In [126]:
tpr0 = np.hstack([tpr0, np.mean(tpr0).round(2)])
tpr1 = np.hstack([tpr1, np.mean(tpr1).round(2)])
tpr2 = np.hstack([tpr2, np.mean(tpr2).round(2)])

tnr0 = np.hstack([tnr0, np.mean(tnr0).round(2)])
tnr1 = np.hstack([tnr1, np.mean(tnr1).round(2)])
tnr2 = np.hstack([tnr2, np.mean(tnr2).round(2)])

accs0 = np.hstack([accs0, np.mean(accs0).round(2)])
accs1 = np.hstack([accs1, np.mean(accs1).round(2)])
accs2 = np.hstack([accs2, np.mean(accs2).round(2)])

In [127]:
mean_tpr = np.mean([tpr0, tpr1, tpr2], axis=0).round(2)
mean_tnr = np.mean([tnr0, tnr1, tnr2], axis=0).round(2)
mean_acc = np.mean([accs0, accs1, accs2], axis=0).round(2)
mean_tpr, mean_tnr, mean_acc

(array([0.98, 0.8 , 0.92, 0.84, 0.58, 0.22, 0.48, 0.87, 0.71]),
 array([0.98, 0.99, 0.7 , 0.98, 1.  , 1.  , 0.99, 0.95, 0.95]),
 array([0.98, 0.98, 0.73, 0.98, 1.  , 1.  , 0.99, 0.95, 0.95]))

In [132]:
ba = (mean_tpr + mean_tnr) / 2
ba

array([0.98 , 0.895, 0.81 , 0.91 , 0.79 , 0.61 , 0.735, 0.91 , 0.83 ])

In [102]:
mean_f1 = np.mean([f1_scores_1, f1_scores_2, f1_scores_3], axis=0).round(2)
mean_f1

array([0.74, 0.65, 0.46, 0.48, 0.46, 0.2 , 0.24, 0.37, 0.45])

In [135]:
df = pd.DataFrame(
    data=np.stack([tpr0, tpr1, tpr2, f1_scores_1, f1_scores_2, f1_scores_3, mean_tpr, mean_tnr, mean_f1, mean_acc, ba]), 
    index=['test0 tpr', 'test1 tpr', 'test2 tpr', 'test0 f1',
           'test1 f1', 'test2 f1', 'mean_tpr', 'mean_tnr', 'mean_f1', 'mean_acc', 'ba'],
    columns=list(range(1,9)) + ['mean']
)

In [136]:
df.to_csv('tpr_newmethod_p0.5.csv')

In [73]:
tpr0.shape

(8,)

In [74]:
f1_scores_1


array([0.80032107, 0.71948766, 0.68360256, 0.46509475, 0.35561878,
       0.3325379 , 0.2945237 , 0.19092164, 0.28375309])